# Milestone 2
#### CS207 Final Project
#### Group 1: _Team Gillet_
#### Lucie Gillet, Jussi Sakari Jukarainen, Jovin Leong, Huahua Zheng


---

# Introduction

<br>

Derivatives play a critical role in the natural and applied sciences, with optimization being one of the core applications involving derivatives. Traditionally, derivatives have been approached either symbolically or through numerical analysis (_e.g._ finite differences). Although numerical approaches to solving derivatives are simple to compute, they are prone to stability issues and round-off errors. Meanwhile, although symbolic derivatives enable the evaluation of derivatives to machine precision, the process is limited by its computational intensity. Recently, the size and complexity of the functions involving derivatives have grown; these demands necessitate an alternative to symbolic and numerical methods that is able to compute derivatives with higher accuracy at a lower cost. Automatic Differentiation (AD) addresses these issues by executing a sequence of elementary arithmetic operations to compute accurate derivatives. 

<br>

Our team aims to develop a Python package, ```superautodiff```, that implements forward-mode AD. We also extend the package to solve simple optimization problems through gradient descent. This document will review some of the the mathematical foundations behind our approach and provide relevant information on documentation and usage of ```superautodiff```.

---

# Background
<br>

## Mathematical Foundations

AD relies heavily on the chain rule and several other key mathematical concepts in order to compute derivatives. We now consider some background mathematical foundations that form the theoretical basis of our approach to AD.

<br>

**Differential calculus**

Differential calculus is concerned with the evaluation and study of gradients and/or rates of change. Numerically, we can formally define the derivative of a function $f$ evaluated at $a$ as:

$$f'(a)=\lim _{h\to 0}{\frac {f(a+h)-f(a)}{h}}$$.


**Elementary functions and their derivatives**

  Here are some examples of elementary functions used by AD and their corresponding derivatives:

  <br>

  **<center> Table 1. Elementary functions and their derivatives  </center>**
  <br>

| $$f(x)$$     | $$f'(x)$$    | 
| ------------- |:-------------:| 
| $$c$$        |         $0$   | 
| $$x$$        |         $1$   | 
| $$x^n$$      | $$nx^{n-1}$$ | 
| $$\frac{1}{x}$$ | $$-\frac{1}{x^2}$$ |
| $$e^x$$      | $$e^x$$ | 
| $$log_ax$$      | $$\frac{1}{x \ln a}$$ | 
| $$\ln x$$      | $$\frac{1}{x}$$ | 
| $$\sin(x)$$      | $$\cos(x)$$ | 
| $$\cos(x)$$      | $$-\sin(x)$$ | 
| $$\tan(x)$$      | $$\frac{1}{\cos^2x}$$ |<br>


<br>

**Chain rule for composite functions**

  The chain rule is a formula used to compute composite derivatives containing multiple variables. For instance, if we have a variable $z$ depending on $y$, which itself depends on $x$, we can subsequently employ the chain rule to express the derivative of $z$ with respect to $x$ is given by:

<br>

$${\frac  {dz}{dx}}={\frac  {dz}{dy}}\cdot {\frac  {dy}{dx}}$$

<br>

**<center> The chain rule </center>**

<br>

**Forward and reverse mode**

  For functions where we have intermediate components in our derivatives, we can keep track of the derivatives of each component using either of the following two modes: the forward mode and the reverse mode.
  -        The forward mode starts with the input and computes the derivative with respect to the input using the chain rule at each subcomponent. The process involves storing the intermediate values of the derivatives of variables with respect to the input in order to evaluate the overall derivative: <br> <br> 
  $$\frac{dw_i}{dx} = \frac{dw_i}{dw_{i-1}}\frac{dw_{i-1}}{dx}$$<br>
   **<center> Forward mode </center>**  
   
<br>
  
  -        The reverse mode, meanwhile, involves both a forward pass that evaluates the values of the functions along with a backward pass that stores the derivatives of the output with respect to the different variables: <br> <br> $$\frac{dy}{dw_i} = \frac{dy}{dw_{i+1}}\frac{dw_{i+1}}{dw_i}$$ <br>    **<center> Reverse mode </center>**

<br>

**Computational graph representation**

  The elementary operations involved in the forward accumulation involved in the forward mode can be visually represented through a computational graph. For instance, the computational graph of the function $f(x)=x−exp\{−2sin^2(4x)\}^{[1]}$ is illustrated on Figure 1; Figure 2 presents a more complex computational graph.

<br>

  The graph breaks down the given function into a sequence of elementary operations that are visually charted out through the computational graph. The graph operates similarly to a flowchart and illustrates how each elementary operation modifies our initial parameter inputs in order to recover the function.


<br>

<img src="fig/graph_1.png" style="height:300px;">

**<center> Figure 1. A computational graph for $f(x)=x−exp\{−2sin^2(4x)\}^{[1]}$</center>**

<br>

<img src="fig/graph_2.png" style="height:450px;">

**<center> Figure 2. A more complex computational graph</center>**

<br>

[1] D. Sondak, lecture 10, CS207 Fall '19

<br>

## What our package is doing
Essentially, our package utilizes the aforementioned mathematical concepts in order to implement the AD through the forward mode. A primary function in our package, ```autodiff()```, takes in mathematical functions and corresponding points at which to evaluate the mathemetical functions and obtains an evaluative trace (similar to that of the graph structure above). Subsequently, this trace is used to perform differentiation on said mathematical function, using the chain rule to evaluate both the derivatives, the derivative values, and the current values at each component of the trace.

Under the hood, we might perceive of the function's calculations as equivalent to populating the table illustrated in Table 2. This is basically the core of forward-mode AD; the functionality and operation of our package is discussed in greater detail in the subsequent section.

<br>

**<center>Table 2. An evaluation table for a foward-mode neural network</center>**

| Trace | Elementary Function | Current Value | Elementary Function Derivative | $\nabla_{x}$ Value  | $\nabla_{y}$ Value  | 
| :---: | :-----------------: | :-----------: | :----------------------------: | :-----------------: | :-----------------: | 
| $x_{1}$ | $x$ | $x$ | $\dot{ x}_{1}$ | $1$ | $0$ |
| $x_{2}$ | $y$ | $y$ | $\dot{x}_{2}$ | $0$ | $1$ |
| $x_{3}$ | $w_{21}x_1$ | $w_{21}x$ | $w_{21}\dot{x}_{1}$ | $w_{21}$ | $0$ |
| $x_{4}$ | $w_{12}x_2$ | $w_{12}y$ | $w_{12}\dot{x}_{2}$ | $0$ | $w_{12}$ |
| $x_{5}$ | $w_{11}x_1$ | $w_{11}x$ | $w_{11}\dot{x}_{1}$ | $w_{11}$ | $0$ |
| $x_{6}$ | $w_{22}x_2$ | $w_{22}y$ | $w_{22}\dot{x}_{2}$ | $0$ | $w_{22}$ |
| $x_{7}$ | $x_4 + x_5$ | $w_{11}x + w_{12}y$ | $$\dot{x}_{4} + \dot{x}_{5}$$ | $w_{11}$ | $w_{12}$ |
| $x_{8}$ | $x_3 + x_6$ | $w_{21}x + w_{22}y$ | $$\dot{x}_{3} + \dot{x}_{6}$$ | $w_{21}$ | $w_{22}$ |
| $x_{9}$ | $z(x_7)$ | $z(w_{11}x + w_{12}y)$ | $$\dot{x}_{7}z'(x_7)$$ | $w_{11}z'(w_{11}x + w_{12}y)$ | $w_{12}z'(w_{11}x + w_{12}y)$ |
| $x_{10}$ | $z(x_8)$ | $z(w_{21}x + w_{22}y)$ | $$\dot{x}_{8}z'(x_8)$$ | $w_{21}z'(w_{21}x + w_{22}y)$ | $w_{22}z'(w_{21}x + w_{22}y)$ |
| $x_{11}$ | $w_{out,1}x_9$ | $$w_{out,1}z(w_{11}x + w_{12}y) $$  | $$w_{out,1}\dot{x}_9$$ | $w_{out,1}w_{11}z'(w_{11}x + w_{12}y)$ | $w_{out,1}w_{12}z'(w_{11}x + w_{12}y)$ |
| $x_{12}$ | $w_{out,2}x_{10}$ | $$w_{out,2}z(w_{21}x + w_{22}y) $$ | $$w_{out,2}\dot{x}_{10}$$ | $w_{out,2}w_{21}z'(w_{21}x + w_{22}y)$ | $w_{out,2}w_{22}z'(w_{21}x + w_{22}y)$ |
| $x_{13}$ | $x_{11} + x_{12}$ | $$w_{out,1}z(w_{11}x + w_{12}y) + w_{out,2}z(w_{21}x + w_{22}y) $$ | $$\dot{x}_{11} + \dot{x}_{12}$$ | $$w_{out,1}w_{11}z'(w_{11}x + w_{12}y) + w_{out,2}w_{21}z'(w_{21}x + w_{22}y)$$ | $$w_{out,1}w_{12}z'(w_{11}x + w_{12}y) + w_{out,2}w_{22}z'(w_{21}x + w_{22}y)$$ |





---

# How to use ```superautodiff```
## User interaction with the package
### Installation
Our package will be distributed throughy PyPI (which will be detailed in the subsequent section). Users will first install the package by running:

```pip install superautodiff -r requirements.txt```

For more advanced users who think they have the required Python dependencies and do not wish to reinstall said dependencies, the following command should be run instead:

```pip install superautodiff```

Users will then need to import the package as in the above use case and will need to import our modules in order to access the package functionalities. Most importantly, users will have to import ```autodiff``` to instantiate AD objects. Subsequently, users will simply have to instantiate their functions and points within the objects in order to perform AD. For the use of the other modules, users will need to import them from our package.

### Importing
After installing the package, users need to subsequently import the various modules into their Python environment. For simplicity's sake, users can just import ```superautodiff``` using the following import command to retrieve all the modules in the package:

```python
import superautodiff
```

Alternatively, it is recommended that users run the following import alias command for concision: 
```python
import superautodiff as sad
```

## Instantiating AD objects

```superautodiff``` is a Python package and its core module is ```autodiff```. Within ```autodiff``` is the ```AutoDiff``` class, where class objects accepts an input $x \in \mathbb{R}$ (stored as the ```val``` attribute) and initializes the derivative (```der``` attribute) at $1$. The ```AutoDiff``` object then supports basic arithmetic operations (_e.g._ addition, multiplication) with integers, floats, and other ```AutoDiff``` objects. These operations will be implemented commutatively through dunder methods as appropriate. With an ```AutoDiff``` object, the user can evaluate the derivatives of a vector of functions at a specified vector of points.
<br><br>



## Usage

We illustrate several use cases of our package's core functionality and show how it can be used to evaluate derivatives for functions about a given point. 

Summarily, the approach that is illustrated below involves importing the module and instantiating an ```AutoDiff``` object. Subsequently, users should use mathematical operations as they see fit in order to map the ```AutoDiff``` object to their target mathematical function.

In [20]:
# Magic command to import autodiff module
%cd ../../cs207-FinalProject
import superautodiff as sad
import math

# Initalize variable inputs and instantiate AutoDiff object
f1 = sad.AutoDiff("x", 3.0)

# Examine initial values
print("Value of f1: {};\nValue of first derivative of f1: {}".format(f1.val, f1.der['x']))

[WinError 2] The system cannot find the file specified: '../../cs207-FinalProject'
C:\Users\jovin\OneDrive\Desktop\CS207\cs207-FinalProject
Value of f1: 3.0;
Value of first derivative of f1: 1.0


In [6]:
# For target function f_a(x) = x**2 + 2x + 3
# We expect the value to be 18 and the value of the derivative to be 8
f1_a = f1 ** 2 + 2 * f1 + 3
print("Value of f1_a: {};\nValue of first derivative of f1_a: {}".format(f1_a.val, f1_a.der['x']))

Value of f1_a: 18.0;
Value of first derivative of f1_a: 8.0


In [8]:
# For target function f_b(x) = cos(πx) + 5x + 4
# We expect the value to be 18 and the value of the derivative to be 5 (approximately)
f1_b = sad.cos(f1 * math.pi) + 5 * f1 + 4
print("Value of f1_b: {};\nValue of first derivative of f1_b: {}".format(f1_b.val, f1_b.der['x']))

Value of f1_b: 18.0;
Value of first derivative of f1_b: 4.999999999999999


In [9]:
# For target function f_c(x) = exp(3x) + 2ln(x) - 12x
# We expect the value to be 8069.2 and the value of the derivative to be 24297.9 (approximately)
f1_c = sad.exp(f1 * 3) + 2 * sad.log(f1) - 12 * f1
print("Value of f1_c: {};\nValue of first derivative of f1_c: {}".format(f1_c.val, f1_c.der['x']))

Value of f1_c: 8069.28115215272;
Value of first derivative of f1_c: 24297.918449392822


Presently, our package has only implemented part of the functionality required for multivariable automatic differentiation because it is not required for Milestone 2.

Once the implementation is complete, we will incorporate more complex multivariable use cases in this section.

---


# Software Organization

## Directory structure



        cs207-FinalProject/
                    LICENSE
                    README.md
                    requirements.txt
                    setup.py
                    setup.cfg
                    travis.yml
                    docs/
                          milestone_1.md
                          milestone_2.md
                          milestone_1.ipynb
                          milestone_2.ipynb
                          fig/
                              graph_1.png
                              graph_2.png
                    superautodiff/
                          __init__.py
                          autodiff.py
                          functions.py
                          optimize.py
                          graddesc.py
                    tests/
                          __init__.py
                          tests_autodiff.py      
                    
                
<br>

## Modules

```superautodiff``` contains four modules corresponding to our package's four main competencies. The modules are summarily described here and explained in detail in the subsequent sections.
- ```autodiff```: This module contains the core functionality of package—a forward mode AD library that is able to work with a vector of input variables for a vector of functions.
- ```function```: This module contains the bulk of the mathematical operations used by our module.
- ```optimize```: This module extends the base functionality of our forward mode AD library by providing functions to solve simple constrained and unconstrained optimization problems.
- ```graddesc```: This module provides functions to perform gradient descent and stochastic gradient descent.

<br>

## Testing
Testing is largely relevant to developers looking to edit and/or build upon our package; general users need not read this section. Our test suite, ``` testsuite.py ```, is stored in our ```tests/``` folder; our testing will be largely monitored through both Travis CI and CodeCov. Our GitHub repository will be fully integrated with Travis CI and CodeCov with relevant badges on our ```README.md``` to reflect the build status on Travis CI and the code coverage status on CodeCov. 

```superautodiff``` also supports ```pytest```. To run our tests, users will need to have ```pytest``` installed on their environment and navigate to the repository. Subsequently, users should run the following code:

```python -m pytest
```

This will run all our tests and provide summary statistics on the outcome of said tests.

<br>

## Package Distribution
Our package is distributed using PyPI. We use _setuptools_ and _wheel_ to generate our distribution archives and we use _twine_ to upload our package to PyPI.

The reason for this choice of tools is that they are simple, easy-to-use, and reliable. Our package does not have many complicated dependencies; we, therefore, want to employ simple packaging and distribution tools to ensure that our package is easily distributed to users with minimal hassle.

As mentioned above, users will simply have to call ```pip install superautodiff``` in order to install our package. The installation instructions and troubleshooting will be available on our GitHub repository.

---

# Implementation 

Thus far, ```superautodiff``` has a working forward mode implementation and we have partly implemented multivariable automatic differentiation.

## Data structures
In our present implementation, the primary data structures used are sets that we use to store our values and derivative values in our ```AutoDiff``` class objects. The reason for this design choice is that we want to prevent cases where we have repeated variables when we implement multivariable automatic differentiation in subsequent milestones. We also use ```Counter``` objects because they enable us to easily store our data in key-value pairs while meshing with our existing use of sets since ```Counter``` objects operate similarly to multisets. The use of both ```Counter``` and sets synergize well in our implementation and ensure consistency across our data structures. 

## Dependencies
Our package relies on the following external packages:
- ```NumPy```: We use this to specify relevant mathematical operations within our package.

- ```collections```: We use this to store our data in ```Counter``` objects.

- ```math```: We use this for additional mathematical operations.

## Dunder methods
The following dunder methods have been overloaded in our implementation in order for our ```AutoDiff``` objects to be easily used in mathematical operations and the construction of mathematical functions:
- ```__add__```: Modified to update the counter objects accordingly when addition is performed; modified to return ```AutoDiff``` objects.

- ```__radd__```: Modified to update the counter objects accordingly when addition is performed; modified to return ```AutoDiff``` objects.

- ```__sub__```: Modified to update the counter objects accordingly when subtraction is performed; modified to return ```AutoDiff``` objects.

- ```__rsub__```: Modified to update the counter objects accordingly when subtraction is performed; modified to return ```AutoDiff``` objects.

- ```__mul__```: Modified to update the counter objects accordingly when multiplication is performed; modified to return ```AutoDiff``` objects.

- ```__rmul__```: Modified to update the counter objects accordingly when multiplication is performed; modified to return ```AutoDiff``` objects.

- ```__neg__```: Modified such that all counter elements are made negative; modified to return ```AutoDiff``` objects.

- ```__truediv__```: Modified such that all counter elements are divided accordingly; modified to return ```AutoDiff``` objects.

- ```__rtruediv__```: Modified such that all counter elements are divided accordingly; modified to return ```AutoDiff``` objects.

- ```__pow__```: Modified such that the counter elements are appropriately exponentiated; modified to return ```AutoDiff``` objects.

## Mathematical operations
Our package implements the following mathematical operations using ```NumPy``` and ```math``` such that they can be used on ```AutoDiff``` objects with ease. All of the following functions can take in scalar values, vectors (Python lists), and ```AutoDiff``` objects. This is useful to users that seek to perform mathematical calculations and/or build up complicated mathematical functions using ```AutoDiff``` objects for derivative evaluation.

### Trigonometric functions
- ```sin(x)```

- ```cos(x)```

- ```tan(x)```

- ```sec(x)```

- ```csc(x)```

- ```cot(x)```

- ```arcsin(x)```

- ```arccos(x)```

- ```arctan(x)```

- ```arcsec(x)```

- ```arccsc(x)```

- ```arccot(x)```

- ```sinh(x)```

- ```cosh(x)```

- ```tanh(x)```


### Logarithmic functions
- ```log(x)```

- ```exp(x)```



Our package defines a class ```autodiff``` that takes a variable ```x``` as input. An ```autodiff``` object has two important attributes: 
- ```val``` - a scalar that contains the value of the function 
- ```der``` - a dictionary that stores the derivatives. For example:

 ```{"a":1, "b":1}```

## Implementation illustrations

### Initialization and instantiation of objects

Once our module is imported, we can create ```AutoDiff``` objects that store the variable name and the value at which to evaluate the variables at. The object is mutable and can undergo mathematical operations in order to create complex mathematical functions; the object stores variable names, the values of the variables (given the value at which to evaluate the variables at), and the values of first derivatives of the variables (given the value at which to evaluate the variables at).

In [30]:
# Import module
import superautodiff as sad

# Initalize variable inputs and instantiate AutoDiff object
value_to_evaluate = 5.0 
variable_name = "x_1"
f1 = sad.AutoDiff(variable_name, value_to_evaluate)

# Illustrate how values and derivative values are stored
print("Value of f1: {};\nValue of first derivative of f1: {}".format(f1.val, f1.der))

Value of f1: 5.0;
Value of first derivative of f1: Counter({'x_1': 1.0})


### Basic operations using dunder methods
The overloaded dunder methods enable the use of basic mathematical operations with ```AutoDiff``` objects. We do not check for the accuracy of our calculations here since that is already covered above in our Usage section; instead, we merely illustrate how the functions are used and the outputs they return in order to showcase our implementation.

In [32]:
# Addition example
f1_a = f1 + f1

# Subtraction example
f1_b = 3*f1 - f1

# Multiplication example
f1_c = f1 * 3

# Exponent example
f1_d = f1 ** 2

# Division example
f1_e = f1/4

print("Value of f1_a: {};\nValue of first derivative of f1_a: {}\n".format(f1_a.val, f1_a.der))
print("Value of f1_b: {};\nValue of first derivative of f1_b: {}\n".format(f1_b.val, f1_b.der))
print("Value of f1_c: {};\nValue of first derivative of f1_c: {}\n".format(f1_c.val, f1_c.der))
print("Value of f1_d: {};\nValue of first derivative of f1_d: {}\n".format(f1_d.val, f1_d.der))
print("Value of f1_e: {};\nValue of first derivative of f1_e: {}\n".format(f1_e.val, f1_e.der))

Value of f1_a: 10.0;
Value of first derivative of f1_a: Counter({'x_1': 2.0})

Value of f1_b: 10.0;
Value of first derivative of f1_b: Counter({'x_1': 2.0})

Value of f1_c: 15.0;
Value of first derivative of f1_c: {'x_1': 3.0}

Value of f1_d: 25.0;
Value of first derivative of f1_d: {'x_1': 10.0}

Value of f1_e: 1.25;
Value of first derivative of f1_e: {'x_1': 0.25}



### Trigonometric and logarithmic operations
Similarly, our ```AutoDiff``` objects can be passed through our trigonometric and logarithmic functions. As before, we do not evaluate check the accuracy of the values as this has already been done above.

In [36]:
# Sine example
f1_f = sad.sin(f1)

# Cosine example
f1_g = sad.cos(f1*2)

# Tangent example
f1_h = sad.tan(f1/2)

# Exp example
f1_i = sad.exp(f1*3)

# Natural logarithm example
f1_j = sad.log(f1+5)

print("Value of f1_f: {};\nValue of first derivative of f1_f: {}\n".format(f1_f.val, f1_f.der))
print("Value of f1_g: {};\nValue of first derivative of f1_g: {}\n".format(f1_g.val, f1_g.der))
print("Value of f1_h: {};\nValue of first derivative of f1_h: {}\n".format(f1_h.val, f1_h.der))
print("Value of f1_i: {};\nValue of first derivative of f1_i: {}\n".format(f1_i.val, f1_i.der))
print("Value of f1_j: {};\nValue of first derivative of f1_j: {}\n".format(f1_j.val, f1_j.der))

Value of f1_f: -0.9589242746631385;
Value of first derivative of f1_f: {'x_1': 0.28366218546322625}

Value of f1_g: -0.8390715290764524;
Value of first derivative of f1_g: {'x_1': 1.0880422217787395}

Value of f1_h: -0.7470222972386602;
Value of first derivative of f1_h: {'x_1': 0.7790211562858627}

Value of f1_i: 3269017.3724721107;
Value of first derivative of f1_i: {'x_1': 9807052.117416332}

Value of f1_j: 2.302585092994046;
Value of first derivative of f1_j: {'x_1': 0.1}



---

# Additional Features


The following are additional features that we plan in include in our subsequent implementations; it is likely that not all features will be included depending on how we want to angle our package, but at this juncture, these are the features we are interested in including in our package.
- Multiple variable inputs: Our package will be extended to take in multiple variables and multiple points at which to evaluate our variables at. This will be accompanied by an update test suite and documentation to reflect this expanded functionality.
- Return Jacboian and Hessian: Our package will have a simple attribute which, when called, is able to return a Jacobian vector containing the first derivatives at each point of evaluation. We also hope to implement a similar attribute that is able to return the Hessian matrix.
- Newton's method: Our module can include a simple algorithm that implements Newton's method using our ```AutoDiff``` objects in order to find the roots. This implementation might only work with single variables - though this is to be determined.
- Additional module for gradient descent and stochastic gradient descent: Similar to our Newton's method algorithm, we hope to implement an additional module for gradient descent and, potentially, stochastic gradient descent that is able to use ```AutoDiff``` objects to find minimum points.